In [1]:
from jdaviz import CubeViz

viz = CubeViz()
viz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [2]:
from astropy.utils.data import download_file

# This file is originally from https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz
# but has been modified to correct some inconsistencies in the way units are parsed
fn = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)
#viz.load_data(fn)

# Begin development code for SpectralCube to Specutils transition

Load Specutils1D cube into jdaviz

In [3]:
from specutils import Spectrum1D
from astropy.io import fits
from spectral_cube.io.fits import FITSReadError
from astropy.io.registry import IORegistryError
from astropy.wcs import WCS
from astropy import units as u
import numpy as np


# temp = Spectrum1D.read(fn)

# print(type(temp))
# print(type(temp.flux))

# viz.app.data_collection["flux"] = temp
# viz.app.data_collection["uncert"] = Spectrum1D(flux=temp.uncertainty)
# viz.app.data_collection["mask"] = temp


# viz.app.add_data_to_viewer('mask-viewer', "mask")

# viz.app.add_data_to_viewer('uncert-viewer', "uncert")

# viz.app.add_data_to_viewer('flux-viewer', "flux")
# viz.app.add_data_to_viewer('spectrum-viewer', "flux")


In [ ]:
temp.flux.unit

In [4]:
with fits.open(fn) as hdulist:
    for hdu in hdulist:
        data_label = f"{fn}[{hdu.name}]"

        if hdu.data is None or not hdu.is_image or hdu.name.lower() not in ["flux", "ivar", "mask"]:
            #print(hdu.name)
            continue
        print(hdu.name)
        wcs = WCS(hdu.header)
#         print("before", wcs)
        wcs.swapaxes(0, 1)
        wcs.swapaxes(1, 2)
        
        if 'BUNIT' in hdu.header:
            flux = hdu.data * u.Unit(hdu.header['BUNIT'])
        else:
            # TODO Add warning that actual units were not found and using fake ones
            flux = hdu.data * u.Jy
        #print(hdu)
        #flux=np.random.sample(10) * u.Jy
        try:
            sc = Spectrum1D(flux=flux, wcs=wcs)
            print("before", sc.wcs)
            #sc = Spectrum1D.read(hdu, format="MaNGA cube")
            sc.wcs = None
#             wcs.swapaxes(0,1)
#             wcs.swapaxes(1, 2)
            sc.wcs = wcs
            print("after", sc.wcs)
            viz.app.data_collection[hdu.name] = sc
        except Exception as e:
            print(e)
            continue
            
        if hdu.name.lower() == "mask":
            viz.app.add_data_to_viewer('mask-viewer', hdu.name)

        if hdu.name.lower() == "ivar":
            viz.app.add_data_to_viewer('uncert-viewer', hdu.name)

        if hdu.name.lower() == "flux":
            viz.app.add_data_to_viewer('flux-viewer', hdu.name)
            viz.app.add_data_to_viewer('spectrum-viewer', hdu.name)
        # sc = Spectrum1D.read(hdu, format="MaNGA cube")
        #print(sc)

FLUX
before WCS Keywords

Number of WCS axes: 3
CTYPE : 'WAVE-LOG'  'DEC--TAN'  'RA---TAN'  
CRVAL : 3.62159598486e-07  27.004754  205.4384  
CRPIX : 1.0  38.0  38.0  
CD1_1 CD1_2 CD1_3  : 8.33903304339e-11  0.0  0.0  
CD2_1 CD2_2 CD2_3  : 0.0  0.000138889  0.0  
CD3_1 CD3_2 CD3_3  : 0.0  0.0  -0.000138889  
NAXIS : 4563  74  74
after WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN'  'DEC--TAN'  'WAVE-LOG'  
CRVAL : 205.4384  27.004754  3.62159598486e-07  
CRPIX : 38.0  38.0  1.0  
CD1_1 CD1_2 CD1_3  : -0.000138889  0.0  0.0  
CD2_1 CD2_2 CD2_3  : 0.0  0.000138889  0.0  
CD3_1 CD3_2 CD3_3  : 0.0  0.0  8.33903304339e-11  
NAXIS : 74  74  4563


/Users/javerbukh/opt/anaconda3/envs/jdaviz-spec-dev/lib/python3.8/site-packages/glue/viewers/common/viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)


0 3 2
axes:  (1, 2)
[flux]
0 3 2
axes:  (1, 2)
[flux]
0 3 2
axes:  (1, 2)
[flux]


IVAR
before WCS Keywords

Number of WCS axes: 3
CTYPE : 'WAVE-LOG'  'DEC--TAN'  'RA---TAN'  
CRVAL : 3.62159598486e-07  27.004754  205.4384  
CRPIX : 1.0  38.0  38.0  
CD1_1 CD1_2 CD1_3  : 8.33903304339e-11  0.0  0.0  
CD2_1 CD2_2 CD2_3  : 0.0  0.000138889  0.0  
CD3_1 CD3_2 CD3_3  : 0.0  0.0  -0.000138889  
NAXIS : 4563  74  74
after WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN'  'DEC--TAN'  'WAVE-LOG'  
CRVAL : 205.4384  27.004754  3.62159598486e-07  
CRPIX : 38.0  38.0  1.0  
CD1_1 CD1_2 CD1_3  : -0.000138889  0.0  0.0  
CD2_1 CD2_2 CD2_3  : 0.0  0.000138889  0.0  
CD3_1 CD3_2 CD3_3  : 0.0  0.0  8.33903304339e-11  
NAXIS : 74  74  4563


/Users/javerbukh/opt/anaconda3/envs/jdaviz-spec-dev/lib/python3.8/site-packages/glue/viewers/common/viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)


MASK
before WCS Keywords

Number of WCS axes: 3
CTYPE : 'WAVE-LOG'  'DEC--TAN'  'RA---TAN'  
CRVAL : 3.62159598486e-07  27.004754  205.4384  
CRPIX : 1.0  38.0  38.0  
CD1_1 CD1_2 CD1_3  : 8.33903304339e-11  0.0  0.0  
CD2_1 CD2_2 CD2_3  : 0.0  0.000138889  0.0  
CD3_1 CD3_2 CD3_3  : 0.0  0.0  -0.000138889  
NAXIS : 4563  74  74
after WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN'  'DEC--TAN'  'WAVE-LOG'  
CRVAL : 205.4384  27.004754  3.62159598486e-07  
CRPIX : 38.0  38.0  1.0  
CD1_1 CD1_2 CD1_3  : -0.000138889  0.0  0.0  
CD2_1 CD2_2 CD2_3  : 0.0  0.000138889  0.0  
CD3_1 CD3_2 CD3_3  : 0.0  0.0  8.33903304339e-11  
NAXIS : 74  74  4563


/Users/javerbukh/opt/anaconda3/envs/jdaviz-spec-dev/lib/python3.8/site-packages/glue/viewers/common/viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)


Why is shape different?

In [9]:
flux = viz.app.get_data_from_viewer('flux-viewer')['FLUX']
print(flux.component_ids())
print(flux.get_object().wcs.get_axis_types())
# flux.main_components
print(temp.wcs.wcs.ctype)
temp.wcs.swapaxes(0,1)
temp.wcs.swapaxes(1, 2)
print(temp.wcs.wcs.ctype)

[Pixel Axis 0 [z], Pixel Axis 1 [y], Pixel Axis 2 [x], Wave, Declination, Right Ascension, flux]
0 3 2
axes:  (1, 2)
[flux]
[{'coordinate_type': 'spectral', 'scale': 'logarithmic', 'group': 0, 'number': 0}]


NameError: name 'temp' is not defined

In [ ]:
with fits.open(fn) as hdulist:
    #print(hdulist[1].header)
    #[print(x, hdulist[1].header[x]) for x in hdulist[1].header]
    
    # print(hdulist[1].header["CTYPE2"])

Alternate way of loading

In [ ]:
with fits.open(fn) as hdulist:
    hdulist = fits.open(fn)
    wcs = WCS(hdulist[1].header)
    flux = hdulist[1].data * u.Unit("Jy")
    print(wcs.world_axis_names)
    #wcs.pixel_n_dim = 3
    #wcs.world_axis_names = ['', '', '']
#     print(wcs)
    sc = Spectrum1D(flux=flux, wcs=wcs)
#     sc.wcs = None
#     sc.wcs = wcs
#     sc.flux.wcs = wcs
    viz.app.data_collection["temp2"] = sc

    
#     print(dir(wcs))

In [ ]:
# viz.load_data(temp)
#temp.wcs.world_axis_names[0].title()
#print(0 in temp.wcs.world_axis_names)
#print(len(temp.wcs.world_axis_names))
temp[52, 52, 2000]

In [ ]:
viz.app.data_collection["temp_flux"] = temp.flux

In [ ]:
temp.shape

In [ ]:
len(temp.wcs.world_axis_names)

In [ ]:
temp.wcs.pixel_n_dim

In [ ]:
temp.ndim = 1

In [ ]:
from glue.core import Data
from glue_astronomy.spectral_coordinates import SpectralCoordinates


In [ ]:
# coords = SpectralCoordinates(temp.wcs)
data = Data(coords=temp.wcs)
data['flux'] = temp.flux.value

In [ ]:

# temp.ndim = lambda: None
# setattr(temp.ndim, 'somefield', 'somevalue')
wcs = temp.wcs
temp.wcs = WCS(wcs)

In [ ]:
from astropy.wcs import WCS

print(isinstance(wcs,WCS))

In [ ]:
print(type(temp.wcs))

In [ ]:
viz.app.data_collection[0].data.get_object().wcs.world_axis_names

In [ ]:
viz.app.data_collection[0].data.get_object().wcs.pixel_n_dim

In [ ]:
conda list